# Cleaning and Analysing Employee Exit Surveys

We'll work with exit surveys from employees of the Department of Education, Training and Employment (DETE) and the Technical and Further Education (TAFE) institute in Queensland, Australia. You can find the DETE exit survey data here. The original TAFE exit survey data is no longer available. We've made some slight modifications to the original datasets to make them easier to work with, including changing the encoding to UTF-8 (the original ones are encoded using cp1252.)

In this project, we'll play the role of data analyst and pretend our stakeholders want to know the following:

- Are employees who only worked for the institutes for a short period of time resigning due to some kind of dissatisfaction? What about employees who have been there longer?
- Are younger employees resigning due to some kind of dissatisfaction? What about older employees?


Below is a preview of a couple columns we'll work with from the dete_survey.csv:

- ID: An id used to identify the participant of the survey
- SeparationType: The reason why the person's employment ended
- Cease Date: The year or month the person's employment ended
- DETE Start Date: The year the person began employment with the DETE

Below is a preview of a couple columns we'll work with from the tafe_survey.csv:

- Record ID: An id used to identify the participant of the survey
- Reason for ceasing employment: The reason why the person's employment ended
- LengthofServiceOverall. Overall Length of Service at Institute (in years): The length of the person's employment (in years)


In [2]:
#import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
#read in both datasets 
dete_survey=pd.read_csv('dete_survey.csv')

tafe_survey=pd.read_csv('tafe_survey.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'dete_survey.csv'

In [ ]:
#basic info about datasets
print(dete_survey.info())
print(tafe_survey.info())

In [ ]:
#check last 5 columns
dete_survey.iloc[:,-5:].isnull()

In the first dataset, dete_survey, we can observe that the majority of columns are filled with some with missing entries in columns (mainly from column 28 onwards). Furthermore, the last 5 rows have an abnormally low number of non-null entries (all less than 40), indicating that either this data is insignificant or that it belongs as a part of another column.

In the second dataset, tafe_survey, we also have many columns that we don't need to complete our analysis. (17-66 are the columns we can exclude)

In [ ]:
dete_survey=pd.read_csv('dete_survey.csv', na_values=['Not Stated'])

In [ ]:
#drop some columns

dete_survey_updated=dete_survey.drop(dete_survey.columns[28:49], axis=1)

tafe_survey_updated=tafe_survey.drop(tafe_survey.columns[17:66], axis=1)

In [ ]:
print(dete_survey_updated.info())
print(tafe_survey_updated.info())

In [ ]:
#rename columns in dete_survey_updated

dete_survey_updated.columns=dete_survey_updated.columns.str.lower().str.strip().str.replace(' ','_')



In [ ]:
print(dete_survey_updated.head())

In [ ]:
#rename collumns in tafe_survey_updated

mapping = {'Record ID': 'id', 'CESSATION YEAR': 'cease_date', 'Reason for ceasing employment': 'separationtype', 'Gender. What is your Gender?': 'gender', 'CurrentAge. Current Age': 'age',
       'Employment Type. Employment Type': 'employment_status',
       'Classification. Classification': 'position',
       'LengthofServiceOverall. Overall Length of Service at Institute (in years)': 'institute_service',
       'LengthofServiceCurrent. Length of Service at current workplace (in years)': 'role_service'}
tafe_survey_updated = tafe_survey_updated.rename(mapping, axis = 1)

In [ ]:
print(tafe_survey_updated.head())

Recall that our end goal is to answer the following question:

- Are employees who have only worked for the institutes for a short period of time resigning due to some kind of dissatisfaction? What about employees who have been at the job longer?


If we look at the unique values in the separationtype columns in each dataframe, we'll see that each contains a couple of different separation types. For this project, we'll only analyze survey respondents who resigned, so their separation type contains the string 'Resignation'.

In [ ]:
print(dete_survey_updated['separationtype'].value_counts())

In [ ]:
print(tafe_survey_updated['separationtype'].value_counts())

In [ ]:
#update different resignation types to just resignation
dete_survey_updated['separationtype'] = dete_survey_updated['separationtype'].str.split('-').str[0]

In [ ]:
#only keep entries with a resignation reason
dete_copy=dete_survey_updated.copy()
tafe_copy=tafe_survey_updated.copy()

#dete_resignations=dete_copy[dete_copy['Resignation-Other reasons', 'Resignation-Other employer', 'Resignation-Move overseas/interstate'] != NaN]

dete_resignations = dete_survey_updated[dete_survey_updated['separationtype'] == 'Resignation'].copy()
tafe_resignations = tafe_survey_updated[tafe_survey_updated['separationtype'] == 'Resignation'].copy()

It may not always be possible to catch all of these errors, but by making sure the data seems reasonable to the best of our knowledge, we can stop ourselves from completing a data analysis project that winds up being useless because of bad data.

In this step, we'll focus on verifying that the years in the cease_date and dete_start_date columns make sense. However, we encourage you to check the data for other issues as well!

Since the cease_date is the last year of the person's employment and the dete_start_date is the person's first year of employment, it wouldn't make sense to have years after the current date.
Given that most people in this field start working in their 20s, it's also unlikely that the dete_start_date was before the year 1940.

In [ ]:
#clean cease_date column in dete_resignations
print(dete_resignations['cease_date'].value_counts())

In [ ]:
dete_resignations['cease_date'] = dete_resignations['cease_date'].str.split('/').str[-1]
dete_resignations['cease_date'] = dete_resignations['cease_date'].astype("float")

dete_resignations['cease_date'].value_counts()

In [ ]:
# Check the unique values and look for outliers
dete_resignations['dete_start_date'].value_counts().sort_values()

In [ ]:
tafe_resignations['cease_date'].value_counts().sort_values()

# Create columns

In [ ]:
dete_resignations['institute_service']=dete_resignations['cease_date']-dete_resignations['dete_start_date']

In [ ]:
print(dete_resignations['institute_service'])

The above column shows how long the employee worked for the company.

# Employee dissatisfaction analysis

Below are the columns we'll use to categorize employees as "dissatisfied" from each dataframe. If you disagree, feel free to modify them! Just make sure you explain why you made that decision.

- tafe_survey_updated:
    - Contributing Factors. Dissatisfaction
    - Contributing Factors. Job Dissatisfaction
- dete_survey_updated:
    - job_dissatisfaction
    - dissatisfaction_with_the_department
    - physical_work_environment
    - lack_of_recognition
    - lack_of_job_security
    - work_location
    - employment_conditions
    - work_life_balance
    - workload


In [ ]:
tafe_resignations['Contributing Factors. Dissatisfaction'].value_counts()

In [ ]:
tafe_resignations['Contributing Factors. Job Dissatisfaction'].value_counts()

In [ ]:
def update_vals(element):
    if pd.isnull(element):
        return np.nan
    elif element=='-':
        return False
    else:
        return True
tafe_resignations['dissatisfied'] = \
tafe_resignations[['Contributing Factors. Dissatisfaction', 'Contributing Factors. Job Dissatisfaction']]\
.applymap(update_vals).any(1, skipna=False)
tafe_resignations_up = tafe_resignations.copy()

In [ ]:
# Check the unique values after the updates
tafe_resignations_up['dissatisfied'].value_counts(dropna=False)

In [ ]:
dissat_dete=['job_dissatisfaction','dissatisfaction_with_the_department','physical_work_environment','lack_of_recognition',\
             'lack_of_job_security','work_location','employment_conditions',\
             'work_life_balance', 'workload']
dete_resignations['dissatisfied']=dete_resignations[dissat_dete].any(1, skipna=False)


In [ ]:
#create copy of dete_resignations
dete_resignations_up=dete_resignations.copy()

In [ ]:
#we're finally ready to combine our datasets! Our end goal \
#is to aggregate the data according to the institute_service column
dete_resignations_up['institute']='DETE'

tafe_resignations_up['institute']='TAFE'

combined=pd.concat([dete_resignations_up, tafe_resignations_up], ignore_index=True)

combined.head()

In [ ]:
#drop columns that are not needed (less than 500 non null values)

combined_updated=combined.dropna(thresh=500, axis =1).copy()

In [ ]:
#clean institute service

combined_updated['institute_service_up']=\
combined_updated['institute_service'].astype(str).str.extract(r'(\d+)')

combined_updated['institute_service_up']=\
combined_updated['institute_service_up'].astype(float)

print(combined_updated['institute_service_up'].value_counts())

In [ ]:
def mapping(el):
    if el<3:
        return 'New: Less than 3 years at a company'
    elif 3<=el and el<=6:
        return 'Experienced: 3-6 years at a company'
    elif 7<=el and el<=10:
        return 'Established: 7-10 years at a company'
    elif el>10:
        return 'Veteran: 11 or more years at a company'
    
combined_updated['service_cat']=combined_updated['institute_service_up'].apply(mapping)



In [ ]:
print(combined_updated['service_cat'])

In [ ]:
combined_updated['dissatisfied'].value_counts(dropna=False)

In [ ]:
combined_updated['dissatisfied']=combined_updated['dissatisfied'].fillna(False)

In [ ]:
#aggregate dissatisfaction values
piv_dissat=combined_updated.pivot_table(index='service_cat', values='dissatisfied')

#plot bar chart
piv_dissat.plot.bar(rot=0)